In [1]:
# The usual fare
# Importing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from astropy.io import fits
import matplotlib.cm as cm
import os 
import glob 
import astropy
import scipy.signal
from astropy.stats import sigma_clip
import time
from astropy import stats
from PIL import Image

In [2]:
cd E:

E:\


In [12]:
# ls

 Volume in drive E is FINKELSTEIN
 Volume Serial Number is 6561-3364

 Directory of E:\

02/04/2019  05:48 PM    <DIR>          20190112
02/04/2019  06:03 PM    <DIR>          20190113
02/04/2019  06:09 PM    <DIR>          20190114
02/05/2019  12:07 PM    <DIR>          20190115
02/04/2019  06:07 PM    <DIR>          20190116
02/05/2019  07:09 PM    <DIR>          20190117
02/05/2019  12:23 PM    <DIR>          20190118
02/05/2019  06:39 PM    <DIR>          20190119
02/05/2019  06:45 PM    <DIR>          20190121
02/05/2019  12:25 PM    <DIR>          20190122
02/05/2019  07:10 PM    <DIR>          20190123
02/05/2019  07:10 PM    <DIR>          20190124
02/05/2019  06:38 PM    <DIR>          focustest
02/06/2019  09:50 PM    <DIR>          In Class Stuffs
02/10/2019  06:06 PM    <DIR>          samplebias_small
               0 File(s)              0 bytes
              15 Dir(s)  92,474,834,944 bytes free


In [3]:
nights = glob.glob('2019*')
nights.sort()
night = nights[0]

In [4]:
cd /20190112/

E:\20190112


In [5]:
all_images = glob.glob('*.fits')
all_images.sort() #get the all_images list in order so indexing is nice

In [40]:
flatdec = '+13:00:03'
biasexptime = '0.001'
IC348dec='+32:03:34' 
ha_filter = '200'
haoff_filter = '202'

In [22]:
def mediancombine(filelist):
    '''
    Edit this docstring accordingly!
    '''
    n = len(filelist)
    first_frame_data = fits.getdata(filelist[0])
    imsize_y, imsize_x = first_frame_data.shape
    fits_stack = np.zeros((imsize_y, imsize_x , n), dtype = np.float32) 
    for ii in range(0, n):
        im = fits.getdata(filelist[ii])
        fits_stack[:,:,ii] = im
    med_frame = np.median(fits_stack, axis=2)
    return med_frame
# still need to median_bias = mediancombine(biasfiles)
# fits.writeto('Backup_Master_Bias.fit', median_bias, biasheader, clobber=True)
# Make Master Bias Path

def filesorter(filename, foldername, fitskeyword_to_check, keyword):
    '''
    This will sort your observation data files for you. It takes in the name of a file,
    the folder you want it in, the keyword in fits to look for, and the keyword category
    in fits to look for the keyword in. Ex. sorting calibrations into new folder --
    filesorter(filename,'calibration','cal','OBJECT')
    '''
    if os.path.exists(filename):
        pass
    else:
        print(filename + " does not exist or has already been moved.")
        return
    
    header = fits.getheader(filename)
    fits_type = header[keyword]
    
    if os.path.exists(foldername):
        pass
    else:
        print("Making new directory: " + foldername)
        os.mkdir(foldername)
        
    if fits_type == fitskeyword_to_check:
        destination = foldername + '/'
        print("Moving " + filename + " to: ./" + destination + filename)
        os.rename(filename, destination + filename)  

    return

def FileMover(filelist,foldername):
        for file in filelist:
            if os.path.exists(file):
                pass
            else:
                print(file + "does not exist or has already been moved")
                return
            if os.path.exists(foldername):
                pass
            else:
                print("Making new directory:" + foldername)
                os.mkdir(foldername)
            destination = foldername + '/'
            print("Moving " + file + " to: ./" + destination + file)
            os.rename(file, destination + file)  


def norm_combine_flats(filelist):
    '''
    This normalizes the flat field images. Each flat will have a 
    pixel value of ~1. Once we do this, we can then median combine
    the flats to get our master flat for a given filter
    '''
    n = len(filelist)
    first_frame_data = fits.getdata(filelist[0])
    imsize_y, imsize_x = first_frame_data.shape
    fits_stack = np.zeros((imsize_y, imsize_x , n), dtype = np.float32) 
    for ii in range(0, n):
        im = fits.getdata(filelist[ii])
        norm_im =  im/np.median(im)# finish new line here to normalize flats
        fits_stack[:,:,ii] = norm_im
    med_frame = np.median(fits_stack, axis=2)
    return med_frame

def CustomBias(filelist):
    for file in filelist:
        im = fits.getdata(file)
        new_name = 'b_'+file[6:-7]+'.fits'
        if im.shape == (4150,4150):
            #im_name = file[6:-7]
            overscan = np.mean(im[:,4100:4140])
            custom_bias = overscan * mmaster_bias
            bias_subtracted = im - custom_bias
            bias_sub_overscan = bias_subtracted[:4110,:4090]
            fits.writeto(new_name,bias_sub_overscan, biasheader,overwrite=True)
            print(new_name,'written.')

#### Making master bias and normalized master bias array

In [8]:
for file in all_images:
        filesorter(file,'bias',0.001,'EXPTIME') # get the biases

Making new directory: bias
Moving c8495t0130b00.fits to: ./bias/c8495t0130b00.fits
Moving c8495t0131b00.fits to: ./bias/c8495t0131b00.fits
Moving c8495t0132b00.fits to: ./bias/c8495t0132b00.fits
Moving c8495t0133b00.fits to: ./bias/c8495t0133b00.fits
Moving c8495t0134b00.fits to: ./bias/c8495t0134b00.fits
Moving c8495t0135b00.fits to: ./bias/c8495t0135b00.fits
Moving c8495t0136b00.fits to: ./bias/c8495t0136b00.fits
Moving c8495t0137b00.fits to: ./bias/c8495t0137b00.fits
Moving c8495t0138b00.fits to: ./bias/c8495t0138b00.fits
Moving c8495t0139b00.fits to: ./bias/c8495t0139b00.fits
Moving c8495t0400b00.fits to: ./bias/c8495t0400b00.fits
Moving c8495t0401b00.fits to: ./bias/c8495t0401b00.fits
Moving c8495t0402b00.fits to: ./bias/c8495t0402b00.fits
Moving c8495t0403b00.fits to: ./bias/c8495t0403b00.fits
Moving c8495t0404b00.fits to: ./bias/c8495t0404b00.fits
Moving c8495t0405b00.fits to: ./bias/c8495t0405b00.fits
Moving c8495t0406b00.fits to: ./bias/c8495t0406b00.fits
Moving c8495t0407b00.

In [9]:
cd bias/

E:\20190112\bias


In [10]:
bias_images=glob.glob('*') # gather bias images
first_bias_data = fits.getdata(bias_images[0]) #
n = len(bias_images) # finds how man bias frames there are so 
                     # we can make the right sized array for them
                     # to stack
imsize_y, imsize_x = first_bias_data.shape
                     # gets the size of the image to make the
                     # right sized array for all the pixel values
biasarray = np.zeros((imsize_y, imsize_x , n), dtype = np.float32)
                     # make an empty array for all our stuff
    
for ii in range(0, n):
    im = fits.getdata(bias_images[ii]) # gets the image data...
    biasarray[:,:,ii] = im # and adds it to a layer of the array
    

biasheader = fits.getheader(bias_images[0]) # gets fits header for bias frames
master_bias = mediancombine(bias_images) # performs median combo
mmaster_bias = master_bias/np.mean(master_bias) # normalizes the master bias for overscan scaling later
print('Normalized Master Bias array generated')

Normalized Master Bias array generated


In [11]:
cd -

E:\20190112


#### Making Master flats

In [21]:
# Finding and sorting Ha Flats
all_images=glob.glob('*.fits')
all_images 
flat_images = []
for flat in all_images:
    if fits.getheader(flat)['DECSTRNG']==flatdec and fits.getheader(flat)['FILTER2 ']==ha_filter:
        flat_images.append(flat)
CustomBias(flat_images)
# Moving Ha flat images
FileMover(flat_images,'HaFlat')
#------------------------------------------
# Bias subtracting Ha Flat images
b_flat_images = glob.glob("b_*.fits")
flat_header= fits.getheader(b_flat_images[0])
# Making Ha master flat
Master_Ha_Flat = norm_combine_flats(b_flat_images)
fits.writeto('MasterHaFlat.fits',Master_Ha_Flat,flat_header,overwrite=True)
print('Master Ha flat Array Generated')
# Moving Ha flat bias subtracted images
FileMover(b_flat_images,'Ha_bias_sub_flats')

b_c8495t0338f.fits written.
b_c8495t0339f.fits written.
b_c8495t0340f.fits written.
b_c8495t0341f.fits written.
b_c8495t0342f.fits written.
Making new directory:HaFlat
Moving HaFlatc8495t0338f00.fits to: ./HaFlat/HaFlatc8495t0338f00.fits
Moving HaFlatc8495t0339f00.fits to: ./HaFlat/HaFlatc8495t0339f00.fits
Moving HaFlatc8495t0340f00.fits to: ./HaFlat/HaFlatc8495t0340f00.fits
Moving HaFlatc8495t0341f00.fits to: ./HaFlat/HaFlatc8495t0341f00.fits
Moving HaFlatc8495t0342f00.fits to: ./HaFlat/HaFlatc8495t0342f00.fits


#### 

In [24]:
# Finding and sorting Ha-off Flats
all_images=glob.glob('*.fits')
all_images 
flat_images = []
for flat in all_images:
    if fits.getheader(flat)['DECSTRNG']==flatdec and fits.getheader(flat)['FILTER2 ']==haoff_filter:
        flat_images.append(flat)
CustomBias(flat_images)
# Moving Ha-off flat images
FileMover(flat_images,'HaOffFlat')
#------------------------------------------
# Bias subtracting Ha-Flat images
b_flat_images = glob.glob("b_*.fits")
b_flat_images.sort()
b_flat_images = b_flat_images[10:14]
b_flat_images
flat_header= fits.getheader(b_flat_images[0])
# Making Ha-off master flat
Master_Haoff_Flat = norm_combine_flats(b_flat_images)
fits.writeto('MasterHaOffFlat.fits',Master_Haoff_Flat,flat_header,overwrite=True)
print('Master Ha off flat Array Generated')
# Moving Ha-off flat bias subtracted images
FileMover(b_flat_images,'HaOff_bias_sub_flats')

b_0336f.fits written.
b_0329f.fits written.
b_0323f.fits written.
b_0335f.fits written.
b_0332f.fits written.
b_0334f.fits written.
b_0327f.fits written.
b_0337f.fits written.
b_0328f.fits written.
b_0326f.fits written.
b_0331f.fits written.
b_0330f.fits written.
b_0333f.fits written.
b_0324f.fits written.
b_0325f.fits written.
Making new directory:HaOffFlat
Moving c8495t0336f00.fits to: ./HaOffFlat/c8495t0336f00.fits
Moving c8495t0329f00.fits to: ./HaOffFlat/c8495t0329f00.fits
Moving c8495t0323f00.fits to: ./HaOffFlat/c8495t0323f00.fits
Moving c8495t0335f00.fits to: ./HaOffFlat/c8495t0335f00.fits
Moving c8495t0332f00.fits to: ./HaOffFlat/c8495t0332f00.fits
Moving c8495t0334f00.fits to: ./HaOffFlat/c8495t0334f00.fits
Moving c8495t0327f00.fits to: ./HaOffFlat/c8495t0327f00.fits
Moving c8495t0337f00.fits to: ./HaOffFlat/c8495t0337f00.fits
Moving c8495t0328f00.fits to: ./HaOffFlat/c8495t0328f00.fits
Moving c8495t0326f00.fits to: ./HaOffFlat/c8495t0326f00.fits
Moving c8495t0331f00.fits to:

#### IC 348 Science Images

In [31]:
# Reglobbing everything
all_images = glob.glob('*fits')
IC348_Ha = []
# Finding IC348 Ha images and adding to empty list
for IC348 in all_images:
    if fits.getheader(IC348)['DECSTRNG']==IC348dec and fits.getheader(IC348)['FILTER2 ']==ha_filter:
        IC348_Ha.append(IC348)
CustomBias(IC348_Ha)
# Moving IC348 Ha science images
FileMover(IC348_Ha,'IC348_Ha')
# Bias subtracting IC348 Ha science images
IC348_Ha_bias_sub = glob.glob('b_*')
# Moving IC348 Ha bias subtracted science images
FileMover(IC348_Ha_bias_sub,'IC348_Ha_bias_sub')
#--------------------------------------------------------------
# Reglobbing everything
all_images = glob.glob('*fits')
IC348_Haoff = []
# Finding IC348 Ha-off images and adding to empty list
for IC348_off in all_images:
    if fits.getheader(IC348_off)['DECSTRNG']==IC348dec and fits.getheader(IC348_off)['FILTER2 ']==haoff_filter:
        IC348_Haoff.append(IC348_off)
CustomBias(IC348_Haoff)
# Moving IC348 Ha science images
FileMover(IC348_Haoff,'IC348_HaOff')
# Moving IC348 Ha bias subtracted science images
IC348_HaOff_bias_sub = glob.glob('b_*')
# Moving IC348 Ha bias subtracted science images
FileMover(IC348_HaOff_bias_sub,'IC348_HaOff_bias_sub')

b_0200o.fits written.
b_0201o.fits written.
b_0202o.fits written.
b_0154o.fits written.
b_0155o.fits written.
b_0156o.fits written.
b_0177o.fits written.
b_0178o.fits written.
b_0179o.fits written.
Making new directory:IC348_Ha
Moving c8495t0200o00.fits to: ./IC348_Ha/c8495t0200o00.fits
Moving c8495t0201o00.fits to: ./IC348_Ha/c8495t0201o00.fits
Moving c8495t0202o00.fits to: ./IC348_Ha/c8495t0202o00.fits
Moving c8495t0154o00.fits to: ./IC348_Ha/c8495t0154o00.fits
Moving c8495t0155o00.fits to: ./IC348_Ha/c8495t0155o00.fits
Moving c8495t0156o00.fits to: ./IC348_Ha/c8495t0156o00.fits
Moving c8495t0177o00.fits to: ./IC348_Ha/c8495t0177o00.fits
Moving c8495t0178o00.fits to: ./IC348_Ha/c8495t0178o00.fits
Moving c8495t0179o00.fits to: ./IC348_Ha/c8495t0179o00.fits
Making new directory:IC348_bias_sub
Moving b_0200o.fits to: ./IC348_bias_sub/b_0200o.fits
Moving b_0201o.fits to: ./IC348_bias_sub/b_0201o.fits
Moving b_0202o.fits to: ./IC348_bias_sub/b_0202o.fits
Moving b_0154o.fits to: ./IC348_bi

In [35]:
# Globbing all the stuff I don't need and moving it out of the way
other_fields = glob.glob('c*.fits')
FileMover(other_fields,'Other_Fields')

Making new directory:Other_Fields
Moving c8495t0050o00.fits to: ./Other_Fields/c8495t0050o00.fits
Moving c8495t0038o00.fits to: ./Other_Fields/c8495t0038o00.fits
Moving c8495t0309o00.fits to: ./Other_Fields/c8495t0309o00.fits
Moving c8495t0199o00.fits to: ./Other_Fields/c8495t0199o00.fits
Moving c8495t0193o00.fits to: ./Other_Fields/c8495t0193o00.fits
Moving c8495t0368f00.fits to: ./Other_Fields/c8495t0368f00.fits
Moving c8495t0369f00.fits to: ./Other_Fields/c8495t0369f00.fits
Moving c8495t0284o00.fits to: ./Other_Fields/c8495t0284o00.fits
Moving c8495t0394f00.fits to: ./Other_Fields/c8495t0394f00.fits
Moving c8495t0358f00.fits to: ./Other_Fields/c8495t0358f00.fits
Moving c8495t0372f00.fits to: ./Other_Fields/c8495t0372f00.fits
Moving c8495t0245o00.fits to: ./Other_Fields/c8495t0245o00.fits
Moving c8495t0196o00.fits to: ./Other_Fields/c8495t0196o00.fits
Moving c8495t0260o00.fits to: ./Other_Fields/c8495t0260o00.fits
Moving c8495t0283o00.fits to: ./Other_Fields/c8495t0283o00.fits
Moving

Moving c8495t0244o00.fits to: ./Other_Fields/c8495t0244o00.fits
Moving c8495t0354f00.fits to: ./Other_Fields/c8495t0354f00.fits
Moving c8495t0356f00.fits to: ./Other_Fields/c8495t0356f00.fits
Moving c8495t0258o00.fits to: ./Other_Fields/c8495t0258o00.fits
Moving c8495t0149o00.fits to: ./Other_Fields/c8495t0149o00.fits
Moving c8495t0346f00.fits to: ./Other_Fields/c8495t0346f00.fits
Moving c8495t0151o00.fits to: ./Other_Fields/c8495t0151o00.fits
Moving c8495t0371f00.fits to: ./Other_Fields/c8495t0371f00.fits
Moving c8495t0042o00.fits to: ./Other_Fields/c8495t0042o00.fits
Moving c8495t0197o00.fits to: ./Other_Fields/c8495t0197o00.fits
Moving c8495t0242o00.fits to: ./Other_Fields/c8495t0242o00.fits
Moving c8495t0143o00.fits to: ./Other_Fields/c8495t0143o00.fits
Moving c8495t0366f00.fits to: ./Other_Fields/c8495t0366f00.fits
Moving c8495t0192o00.fits to: ./Other_Fields/c8495t0192o00.fits
Moving c8495t0161o00.fits to: ./Other_Fields/c8495t0161o00.fits
Moving c8495t0290o00.fits to: ./Other_Fi

In [36]:
cd IC348_Ha_bias_sub/

E:\20190112\IC348_Ha_bias_sub


In [39]:
IC348_Ha_bias_sub = glob.glob('*.fits')
for image in IC348_Ha_bias_sub:
    data = fits.getdata(image)
    header = fits.getheader(image)
    flat_fielded = data / Master_Ha_Flat
    name = 'f_' + image
    fits.writeto(name,flat_fielded,header,overwrite=True)
    print(name + ' has been flatfielded')

f_b_0200o.fits has been flatfielded
f_b_0201o.fits has been flatfielded
f_b_0202o.fits has been flatfielded
f_b_0154o.fits has been flatfielded
f_b_0155o.fits has been flatfielded
f_b_0156o.fits has been flatfielded
f_b_0177o.fits has been flatfielded
f_b_0178o.fits has been flatfielded
f_b_0179o.fits has been flatfielded


In [42]:
cd -

E:\20190112


In [43]:
cd IC348_HaOff_bias_sub/

E:\20190112\IC348_HaOff_bias_sub


In [44]:
IC348_Haoff_bias_sub = glob.glob('*.fits')
for image in IC348_Haoff_bias_sub:
    data = fits.getdata(image)
    header = fits.getheader(image)
    flat_fielded = data / Master_Haoff_Flat
    name = 'f_' + image
    fits.writeto(name,flat_fielded,header,overwrite=True)
    print(name + ' has been flatfielded')

f_b_0182o.fits has been flatfielded
f_b_0203o.fits has been flatfielded
f_b_0204o.fits has been flatfielded
f_b_0205o.fits has been flatfielded
f_b_0157o.fits has been flatfielded
f_b_0158o.fits has been flatfielded
f_b_0159o.fits has been flatfielded
f_b_0180o.fits has been flatfielded
f_b_0181o.fits has been flatfielded


#### HL Tau Science Images